In [1]:
%load_ext autoreload
%autoreload 2
import spacy
import scispacy
from scispacy.linking import EntityLinker
import en_core_sci_lg
from scify.nlp import *
from scify.utils.GNBR_preprocessing import get_data_and_distributions

UMLS is really big. Better to load linkers in separate cells. Sometimes crashes or similar on my Macbook Air

In [2]:
# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
# NOTE: The resolve_abbreviations parameter is optional, and requires that
# the AbbreviationDetector pipe has already been added to the pipeline. Adding
# the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# that linking will only be performed on the long form of abbreviations.

#UMLS is big and takes a while EVEN when CACHED it is downloading from tfidf_vectors from here idk why:
#url: /ai2-s2-scispacy/data/umls/tfidf_vectors_sparse.npz
linker_umls = EntityLinker(resolve_abbreviations=True, name="umls")

#linker_go = EntityLinker(resolve_abbreviations=True, name="go")
#linker_mesh = EntityLinker(resolve_abbreviations=True, name="mesh")

/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [3]:
#linkers = [EntityLinker(resolve_abbreviations=True, name=ontology) for ontology in ["umls", "go", "mesh"]] this might crash
#linkers = [linker_mesh, linker_go, linker_umls]
#[get_ontology_name(linker) for linker in linkers]

#NER_models = [en_ner_bc5cdr_md, en_ner_craft_md, en_ner_bionlp13cg_md, en_ner_jnlpba_md]
#nlps = [model.load() for model in NER_models]

In [4]:
#Specialized NER models trained on specialized corpus
import en_ner_bc5cdr_md
import en_ner_craft_md
import en_ner_bionlp13cg_md
import en_ner_jnlpba_md

In [5]:
text="Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily."

I'm loading models here in every cell (instead of mapping them) separately because it sometimes crashes...

In [6]:
base_nlp = spacy.load("en_core_sci_lg")

In [7]:
nlp1 = spacy.load("en_ner_bc5cdr_md")

In [8]:
nlp2 = spacy.load("en_ner_craft_md") #craft has loading issues? buggy....

In [9]:
nlp3 = en_ner_bionlp13cg_md.load()

In [10]:
nlp4 = en_ner_jnlpba_md.load()

In [11]:
NER_nlps = [nlp1, nlp2, nlp3, nlp4]

In [12]:
nlps = add_pipes_mutative(NER_nlps, linker_umls)

In [13]:
base_nlp.pipeline, nlps[3].pipeline, nlps[0].meta["name"]

([('tagger', <spacy.pipeline.pipes.Tagger at 0x1b1913f70>),
  ('parser', <spacy.pipeline.pipes.DependencyParser at 0x1252e72e0>),
  ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x1252e7040>)],
 [('tagger', <spacy.pipeline.pipes.Tagger at 0x2f95a25e0>),
  ('parser', <spacy.pipeline.pipes.DependencyParser at 0x32041fca0>),
  ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x2c709a760>),
  ('AbbreviationDetector',
   <scispacy.abbreviation.AbbreviationDetector at 0x32e2bec10>),
  ('merge_entities', <function spacy.pipeline.functions.merge_entities(doc)>),
  ('EntityLinker', <scispacy.linking.EntityLinker at 0x10eab7400>)],
 'ner_bc5cdr_md')

Be Very careful to overwrite Ents .... there's many attributes that are needed! I just attach an extension
(but in the end, my algorithm just looks at labels_ to determine if a sentence is worthy of pattern matching!)

1) _token the product(NER modules x mesh/go/umls)
2) Read the Text with all of them
3) Extract their labels and ids + confidence scores
4) Look at set differences -> overlap intersect?
4) Add labels to doc @ Scispacy_lg (no NER) so it has Labels (set_extension)
5) Assign highest confidence label to scispacy large

In [14]:
#linker_umls.kb.cui_to_entity["5372017"]
#D016472 is a mesh ID

In [15]:
chem_gene_in_texts = [
    "The antiarrhythmic effect of carvedilol was examined in a placebo-controlled multicenter trial , the Carvedilol Post-Infarct Survival Control in Left_Ventricular_Dysfunction -LRB- CAPRICORN -RRB- study , which enrolled 1,959 patients with reduced left ventricular systolic function after AMI , 98 % of whom were treated with an ACE inhibitor .",
   "Carvedilol improves survival of patients suffering from CHF but the effects of the drug on angiotensin-converting_enzyme -LRB- ACE -RRB- activity , renin and aldosterone are not well characterized in patients receiving an ACE inhibitor .",
"We therefore assessed the effects of introducing carvedilol and bisoprolol in a prospective manner on indices of oxidative stress -LSB- lipid hydroperoxides -LRB- LHP -RRB- -RSB- , endothelial damage -LSB- von_Willebrand factor -LRB- vWf -RRB- -RSB- , platelet activation -LRB- soluble P-selectin -RRB- and coagulation -LRB- fibrinogen -RRB- and their inter-relationships in stable outpatients with CHF in sinus rhythm ."
]

In [ ]:
def _rerun_GNBR_annotation():
    #already in datasets
        docs = get_merged_docs_for_texts(chem_gene_in_texts, base_nlp, nlps)
        [doc for doc in docs if doc_has_entity_labels(doc,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]
        [annotation["label"] 
         for ent in docs[0].ents
         for annotation in ent._.annotated]

In [18]:
from scify.nlp import construct_pattern, prep_pattern

In [19]:
construct_pattern(prep_pattern("kinase|compound|START_ENTITY RAFTK|nsubj|kinase RAFTK|dep|is is|nsubj|kinase kinase|nmod|activation activation|nmod|END_ENTITY"))

In [20]:
sent1 = """In this study , we report that the related adhesion focal tyrosine kinase RAFTK , is an upstream kinase in beta1 integrin mediated activation of Akt."""

In [21]:
visualise_doc(base_nlp(sent1))

In [22]:
#get_non_zero support for one of the 30 themes only
#dependency paths + support
#Maybe merge dependency paths
#distributions

#it's very big but looks like this
distributions_kv_example = {"kinases|compound|START_ENTITY participate|nsubj|kinases participate|nmod|END_ENTITY": {'A+': 2.0,
  'A+.ind': 0.0,
  'A-': 0.0,
  'A-.ind': 0.0,
  'B': 0.0,
  'B.ind': 0.0,
  'E+': 0.0,
  'E+.ind': 0.0,
  'E-': 0.0,
  'E-.ind': 0.0,
  'E': 0.0,
  'E.ind': 0.0,
  'N': 0.0,
  'N.ind': 0.0,
  'O': 2.0,
  'O.ind': 0.0,
  'K': 0.0,
  'K.ind': 0.0,
  'Z': 0.0,
  'Z.ind': 0.0}}

In [23]:
from scify.nlp import rgetattr, rsetattr
linker_umls.kb.cui_to_entity["C1413931"]

CUI: C1413931, Name: ACE gene
Definition: This gene plays a role in vasoconstriction.
TUI(s): T028
Aliases (abbreviated, total: 19): 
	 ACE gene, ACE gene, ACE Gene, ACE Gene, ANGIOTENSIN I-CONVERTING ENZYME, angiotensin I converting enzyme, KININASE II, peptidyl-dipeptidase A, ACE, ACE

In [25]:
data, distributions = get_data_and_distributions()

['A+', 'A+.ind', 'A-', 'A-.ind', 'B', 'B.ind', 'E+', 'E+.ind', 'E-', 'E-.ind', 'E', 'E.ind', 'N', 'N.ind', 'O', 'O.ind', 'K', 'K.ind', 'Z', 'Z.ind'] headers 
        chemical-gene
(A+) agonism, activation
(A-) antagonism, blocking
(B) binding, ligand (esp. receptors)
(E+) increases expression/production
(E-) decreases expression/production
(E) affects expression/production (neutral)
(N) inhibits
        


In [26]:
sents = [d["sent"] for d in data]

In [27]:
len(sents)
sent_sample = sents[:500]

In [28]:
docs = get_merged_docs_for_texts(sent_sample, base_nlp, nlps)

Merging Named Entities (Chems, Gene, Organism etc.). Later docs in the pipeline overwrite entities from earlier ones


In [29]:
wanted_sents = [doc for doc in docs if doc_has_entity_labels(doc,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]

Let's see how well our NER modules work. Theoretically every sentence in the data should have at least one chemical and one gene entity in it

Ok that's 40% coverage....that's kinda bad

In [30]:
len(wanted_sents), wanted_sents[200]


(207,
 MAIN OUTCOMES MEASURES : Progesterone , 17-hydroxyprogesterone , 11-deoxycorticosterone , corticosterone , aldosterone , 11-deoxycortisol , cortisol , cortisone , dehydroepiandrosterone_sulfate -LRB- DHEAS -RRB- , androstenedione , T , dihydrotestosterone , insulin resistance index of the homeostasis model assessment , and blood pressure were measured .)

Let's check for patterns


In [31]:
patterns = [d["dep"] for d in data[:500]]

constructed = [construct_pattern(pattern) for pattern in patterns]

len([con for con in constructed if con != None])

430

I can live with that ratio 430/500 (maybe errors not evenly distributed)

In [32]:
import os
os.listdir("../data/pubmed")

['sample_pubmed20n0004.xml',
 'draw.io-13.5.1.dmg',
 '.DS_Store',
 'pubmed20n0008.xml.gz',
 'pubmed20n0004.xml.gz',
 'pubmed20n0004.xml',
 'pubmed20n0010.xml',
 'pubmed20n0001.xml',
 'pubmed20n0010.xml.gz',
 'LB4-Linguamatics-Brochure-Life-Science-overview.pdf',
 'J_Entrez.txt',
 'Zusammenfassung.pdf',
 'pubmed20n0008.xml',
 '1908.10784.pdf',
 'pubmed20n0001.xml.gz',
 'pubmed20n1015.xml',
 'sample_pubmed_1.xml']

In [33]:
from pubmed_parser import parse_medline_xml
pubmed_abstracts = parse_medline_xml("../data/pubmed/pubmed20n1015.xml")
abstr = [article["abstract"] for article in pubmed_abstracts]

In [35]:
#this took an hour on my MacbookAir 2020
docs_pubmed = get_merged_docs_for_texts(abstr, base_nlp, nlps)

Merging Named Entities (Chems, Gene, Organism etc.). Later docs in the pipeline overwrite entities from earlier ones
index : 18 --- error for doc: Renal oste
index : 21 --- error for doc: Background
index : 32 --- error for doc: Mesenchyma
index : 42 --- error for doc: Introducti
index : 48 --- error for doc: Introducti
index : 48 --- error for doc: Introducti
index : 62 --- error for doc: Background
index : 62 --- error for doc: Background
index : 81 --- error for doc: Background
index : 87 --- error for doc: Mutations 
index : 89 --- error for doc: Correlatio
index : 96 --- error for doc: Objective:
index : 99 --- error for doc: Objective:
index : 99 --- error for doc: Objective:
index : 99 --- error for doc: Objective:
index : 112 --- error for doc: Arabinoman
index : 130 --- error for doc: Background
index : 138 --- error for doc: During pre
index : 139 --- error for doc: Intraventr
index : 140 --- error for doc: We present
index : 140 --- error for doc: We present
index : 175 --- 

In [36]:
from textacy.corpus import Corpus, _get_spacy_lang

In [37]:
corp = Corpus(base_nlp, docs_pubmed)

In [58]:
corp.docs[3].ents

(Objective,
 report,
 surgical techniques,
 results,
 treatment,
 chronic Monteggia fracture-dislocation,
 children,
 pediatric,
 patients,
 procedure,
 surgical steps,
 retrospectively,
 evaluated,
 extended,
 lateral,
 fibrotic,
 removal,
 3-proximal ulnar osteotomy,
 reduction,
 radial head and transcapitellar temporary fixation,
 ulnar fixation,
 straight plate,
 deformity,
 temporary,
 fixation,
 6- transcapitellar Kirschner wire,
 patients,
 women,
 right-sided compromise,
 patients,
 years,
 minimum,
 follow-up,
 period,
 months,
 mean time,
 onset of,
 fracture,
 treatment,
 months,
 patients,
 complete,
 patient,
 prono-supination,
 patients,
 loss of pronation,
 patient,
 decrease,
 supination,
 redislocation,
 radial head,
 follow-up,
 evaluation,
 complications,
 complaint,
 salience,
 ulnar plate,
 results,
 effective,
 treatment,
 chronic Monteggia fracture-dislocation,
 children,
 small study,
 technical,
 surgical strategies)

In [57]:
corp.save("../data/experiments/pubmed_20n1015_annotated_756")



In [92]:
a = [1,2,3]
a.extend([4])
a

[1, 2, 3, 4]

In [82]:
corp= Corpus(base_nlp).load(base_nlp, "../data/experiments/GNBR_chem-gene_annotated_500")
#Doc.set_extension("annotated", default=None)
#doc = nlp(text)

In [113]:
def count_annotated_named_entities_in_docs(docs):
    annotations = []
    for doc in docs:
        for ent in doc.ents:
            annotations.extend(ent._.annotated)
    return annotations


In [118]:
from collections import Counter


In [119]:

Counter(info["label"] for info in count_annotated_named_entities_in_docs(docs_pubmed))

Counter({'CHEBI': 1626,
         'DISEASE': 2401,
         'ORGANISM': 1731,
         'MULTI-TISSUE_STRUCTURE': 617,
         'CHEMICAL': 1775,
         'SIMPLE_CHEMICAL': 1768,
         'ORGANISM_SUBSTANCE': 172,
         'TAXON': 703,
         'GENE_OR_GENE_PRODUCT': 1993,
         'PROTEIN': 2148,
         'CELL': 636,
         'GGP': 1077,
         'ORGAN': 527,
         'DNA': 531,
         'TISSUE': 475,
         'PATHOLOGICAL_FORMATION': 94,
         'CANCER': 781,
         'SO': 752,
         'CL': 315,
         'CELL_TYPE': 308,
         'CELL_LINE': 129,
         'GO': 184,
         'ORGANISM_SUBDIVISION': 49,
         'CELLULAR_COMPONENT': 192,
         'RNA': 28,
         'IMMATERIAL_ANATOMICAL_ENTITY': 32,
         'ANATOMICAL_SYSTEM': 1,
         'AMINO_ACID': 1,
         'DEVELOPING_ANATOMICAL_STRUCTURE': 1})

In [108]:
docs_pubmed[1].ents[10]._.annotated #._. was only set on entities

[{'idx': 0,
  'kb_ents': [('C1846821', 0.9105145931243896),
   ('C0151563', 0.7594596743583679),
   ('C4703602', 0.7459331750869751),
   ('C0441509', 0.7311907410621643),
   ('C1328723', 0.7311907410621643)],
  'label': 'DISEASE',
  'model': 'umls',
  'end_char': 257,
  'start_char': 237}]

In [86]:
base_nlp.meta

{'lang': 'en',
 'name': 'core_sci_lg',
 'version': '0.2.5',
 'spacy_version': '>=2.3.0',
 'description': 'Spacy Models for Biomedical Text.',
 'author': 'Allen Institute for Artificial Intelligence',
 'email': 'ai2-info@allenai.org',
 'url': 'https://allenai.github.io/SciSpaCy/',
 'license': 'CC BY-SA 3.0',
 'vectors': {'width': 200,
  'vectors': 780000,
  'keys': 667874,
  'name': 'en_core_sci_lg_vectors'},
 'pipeline': ['tagger', 'parser', 'ner'],
 'factories': {'tagger': 'tagger', 'parser': 'parser', 'ner': 'ner'},
 'labels': OrderedDict([('tagger',
               ['$',
                "''",
                ',',
                '-LRB-',
                '-RRB-',
                '.',
                ':',
                'ADD',
                'AFX',
                'CC',
                'CD',
                'DT',
                'EX',
                'FW',
                'HYPH',
                'IN',
                'JJ',
                'JJR',
                'JJS',
               

In [111]:
corp[2].ents, corp[2].vocab, base_nlp.vocab

((decreased,
  cholinesterase,
  inhibition,
  brain,
  i.p,
  injection,
  parathion/kg body weight,
  equitoxic,
  active,
  metabolite,
  paraoxon -LRB- 0,0-diethyl_0-4-nitrophenyl_phosphate),
 <spacy.vocab.Vocab at 0x232d1a540>)

In [49]:
most_frequent_words = {base_nlp.vocab[k].text: v for k, v in sorted(corp.word_counts().items(), key=lambda item: item[1], reverse=True)}
5

In [59]:
wanted_abstr = [doc for doc in docs_pubmed if doc_has_entity_labels(doc,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]

In [62]:
target_sents =[sent for abstract in wanted_abstr for sent in abstract.sents if doc_has_entity_labels(sent,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]

len(wanted_abstr), len([sent for abstract in wanted_abstr for sent in abstract.sents ]), len(target_sents)

(296, 3022, 491)

In [81]:
target_sents[21][11]

spacy.tokens.token.Token

In [70]:
visualise_doc(target_sents[24])